<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Орбиты в геометрии Шварцшильда (Общая теория относительности)

### Examples - Astrophysics
<section class="post-meta">
By Eilif Sommer Øyre, Jonas Tjemsland and Jon Andreas Støvneng
</section>
Last edited: January 25th 2018 
___

В этом блокноте мы обсудим орбиты в общей теории относительности. Более конкретно, мы рассмотрим орбиты в геометрии Шварцшильда, названной в честь Карла Шварцшильда (1873-1916), который решил уравнение Эйнштейна в 1916 году [[1]](#rsc). Эта геометрия описывает кривизну пространства-времени вокруг сферически симметричного распределения массы. Это в превосходном приближении соответствует искривленному пространству-времени вне Солнца. Однако релятивистские поправки к ньютоновской гравитации ничтожны. Поэтому мы рассмотрим космический корабль, планету или звезду, вращающуюся вокруг массивной черной дыры.

Можно показать (см. [приложение](#apnd)), что сила, действующая на частицу массой $m\ll M$ вне сферически симметричного распределения массы массы $M$, задается

\begin{equation}
\mathbf F =-\frac{GMm}{r^3}\mathbf r\left(1 + \frac{3l^2}{c^2r^2}\right),
\label{eq:force} \quad(1)
\end{equation}

где $G$ - ньютоновская гравитационная постоянная, $r$ - расстояние до центра распределения массы, $l$ - угловой момент на единицу массы и $c$ - скорость света.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import time
%matplotlib inline

In [ ]:
# Set common figure parameters
fig_size = 10
newparams = {'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

### Единицы

Мы будем использовать натуральные единицы, т.е. Пусть $G=c=1(=\hbar=k_B)$. В этих единицах любая скорость задается в терминах скорости света. То есть, если космический корабль имеет скорость $v=0.1$ в натуральных единицах, он имеет скорость $v=0.1 c\approx 3\cdot 10^7\,\mathrm{m/s}$ в единицах СИ.

Мы будем измерять расстояния в терминах радиуса Шварцшильда, часто называемого "горизонтом событий" или "точкой невозврата" в случае Черных дыр; см. [приложение](#apnd). Он задается

\begin{equation}
R_\mathrm{S} = \frac{2MG}{c^2} = 2M.
\end{equation}

Ничто не может убежать, оказавшись в радиусе Шварцшильда, даже свет. Мы не будем обсуждать это дальше, но оставим пару упражнений на эту тему позже. Введем безразмерные переменные

\begin{equation}
\rho\equiv \frac{r}{R_S} = \frac{r}{2M_\odot\mu} \quad \mathrm{and}\quad T\equiv \frac{\tau}{t_0},
\end{equation}

где $t_0$ - некоторое характерное время системы, а $\tau$ - собственное время. Обратите внимание, что $t_0$ имеют  размерность массы. Таким образом, мы можем выбрать $t_0=R_\mathrm{S}$.

**Упражнение:** Чему равен $R_\mathrm{S}$ для Солнца в единицах СИ? Чему равен $t_0$ в единицах СИ?  
*Ответ:* $\sim 3\,\mathrm{km}$, $\sim 10\,\mathrm{\mu s}$

### Эффективный Потенциал. Различия между классическими и орбитами в ОТО

В приложении мы покажем, что система может быть описана уравнением

\begin{equation}
\mathcal E = \frac{1}{2}\left(\frac{\mathrm d r}{\mathrm d\tau}\right)^2 + V_\mathrm{eff}(r),
\label{eq:energy_ein} \quad(2)
\end{equation}

где $\mathcal E$ - константа и

\begin{equation}
V_\mathrm{eff}(r)\equiv -\frac{M}{r}+\frac{l^2}{2r^2}- \frac{Ml^2}{r^3}
\label{eq:effpot} \quad(3)
\end{equation}

это эффективный потенциал. Константа $\mathcal E$ сводится в классическом пределе к плотности энергии частицы. Обратите внимание, что эффективный потенциал имеет точки экстремума в

\begin{equation}
r_\mathrm{min/max} = \frac{l^2}{2m}\mp \frac{l}{2}\sqrt{\frac{l^2}{m^2}-12}.
\end{equation}

В классической механике эффективный потенциал задается

\begin{equation}
V_\mathrm{classical}(r)= -\frac{M}{r}+\frac{l^2}{2r^2}.
\end{equation}

Этот потенциал имеет минимум при $r = l^2/M$

Последний член в уравнении (3) является общей релятивистской поправкой. Это слагаемое дает несколько различных типов орбит, с которыми мы не сталкиваемся в классической механике. Чтобы увидеть это, мы сначала визуализируем потенциалы. В приведенном ниже коде мы использовали вновь определенные единицы измерения.

In [ ]:
def Veff(rho, l):
    """ Оценивает эффективный потенциал в геометрии Шварцшильда. """
    return -1/(2*rho) + l**2/(2*rho**2) - l**2/(2*rho**3)

def Vclassical(rho, l):
    """ Оценивает классический эффективный потенциал. """
    return -1/(2*rho) + l**2/(2*rho**2)

def K(Z):
    """ Оценивает кинетическую часть плотности энергии. 
    Z = (x, y, u, v). 
    """
    rho2 = Z[0, :]**2 + Z[1, :]**2
    return .5 * (Z[3, :]*Z[1, :] + Z[2, :]*Z[0, :])**2/rho2

In [ ]:
N = 1000
rho = np.linspace(1, 30, N)
l = 2

# Оценить потенциал
VGR = Veff(rho, l)
VCM = Vclassical(rho, l)

# Max/min
# Классическая механика
rhoCM_min = 2*l**2
VCM_min = -0.125/l**2
# Общая теория относительности
rho_min = l**2 + l*(l**2 - 3)**.5
VGR_min = Veff(rho_min, l)
rho_max = l**2 - l*(l**2 - 3)**.5
VGR_max = Veff(rho_max, l)

plt.figure(figsize=(fig_size, fig_size/2))
# Потенциалы
plt.plot(rho, VGR, label="GR")
plt.plot(rho, VCM, label="CM")
# Три различных типа орбит
edge = VGR_max - VGR_min
plt.plot([0, rho[-1]], 2*[VGR_max + edge/6], "--m", label="Orbit 1")
plt.plot([0, rho[-1]], 2*[(VGR_max + VGR[-1])/2], "-.m", label="Orbit 2")
plt.plot([0, rho[-1]], 2*[(VCM_min + VGR[-1])/2], ":m", label="Orbit 3")
# Экстремум
plt.plot(rhoCM_min, VCM_min, 'ko', label="Extremum")
plt.plot(rho_min, VGR_min, 'ko')
plt.plot(rho_max, VGR_max, 'ko')
# Настройки осей
plt.ylabel(r"$V$")
plt.xlabel(r"$\rho=r/R_S$")
plt.ylim([VGR_min - edge/3, VGR_max + edge/3])
plt.xlim([0, rho[-1]])
plt.legend(loc=1)
plt.show()

В классической механике существует только два различных типа орбит:
- связанные орбиты: эллипсы и круги (орбиты 2 и 3)
- орбиты рассеяния: гипербола (орбита 1)

В классической механике существует только одна круговая орбита, и она стабильна.

В общей теории относительности существует три типа орбит:
- радиальное погружение (орбита 1)
- орбиты рассеяния (орбита 2)
- связанные орбиты: прецессирующие эллипсы (орбита 3)

Обратите внимание, что в общей теории относительности существует две круговые орбиты: одна нестабильная (максимальная) и одна стабильная (минимальная). Связанные орбиты представляют собой либо круги, либо прецессирующие эллипсы. В случае рассеивающих орбит, особенно вблизи максимума, орбита может несколько раз опоясать черную дыру, прежде чем уйти в бесконечность. Мы встретимся с тремя различными орбитами в качестве примеров позже в этом блокноте. 

Для более полного аналитического анализа мы отсылаем вас к ссылке [[1]](#rsc).

### Уравнения движения и численная реализация

Используя второй закон Ньютона и недавно введенные переменные, мы можем переписать уравнение (1) как

\begin{equation}
\frac{\mathrm{d}^2 \rho}{\mathrm{d}T^2} =-\frac{1}{2\rho^2}\left(1 + \frac{3l^2}{\rho^2}\right),
\label{eq:eqmotion} \quad(4)
\end{equation}

где мы переопределили $l$ как $l/R_s$.

Классические орбиты расположены в плоскости из-за сохранения углового момента. То же самое верно и для орбит в геометрии Шварцшильда (см. Приложение). Таким образом, мы можем выбрать, что частица движется в плоскости $xy$.

Пусть $\mathrm \rho \equiv (x, y)$, $u\equiv \partial x/\partial \tau$ и $v\equiv \partial y/\partial \tau$. Мы можем записать (в натуральных единицах) уравнение (1) в виде четырех связанных дифференциальных уравнений первого порядка 

\begin{equation}
\frac{\mathrm{d}x}{\mathrm{d}T}=u,\quad \frac{\mathrm{d}u}{\mathrm{d}T}=-\frac{x}{2\rho^3}\left(1 + \frac{3l^2}{\rho^2}\right),
\end{equation}

\begin{equation}
\frac{\mathrm{d}y}{\mathrm{d}T}=v,\quad \frac{\mathrm{d}v}{\mathrm{d}T}=-\frac{y}{2\rho^3}\left(1 + \frac{3l^2}{\rho^2}\right).
\end{equation}

Обратите внимание, что $u$ и $v$ заданы как доли скорости света, что означает, что $|u|\leq 1$, $|v|\leq 1$.

Мы реализуем метод Рунге-Кутты четвертого порядка. В приведенном ниже коде мы определяем $A\equiv 1/2$ и $B\equiv 3l^2$

In [ ]:
def getB(x, y, u, v):
    """ Вычисляет константу B. """
    l2 = (x*v - y*u)**2
    # Note that l2 has the "dimension" R_s 
    return 3*l2

def getA():
    """ Вычисляет константу A. """
    return .5

def RHS(Z, A, B):
    """ Возвращает производные по времени от Z = X, Y, U и V. """
    
    rho = np.sqrt(Z[0]**2 + Z[1]**2)
    correction = 1 + B/rho**2
    dUdtau = -A*Z[0]/rho**3*correction
    dVdtau = -A*Z[1]/rho**3*correction
    
    return np.array([Z[2], Z[3], dUdtau, dVdtau])

def rk4step(f, y, h, A, B):
    """Calculate next step of an IVP of an ODE with a RHS described by the RHS function with RK4.
    Parameters:
        f: function. Right hand side of the ODE.
        y: float. Current step (position).
        h: float. Step-length.
    Returns:
        Next step.
    """
    
    s1 = f(y, A, B)
    s2 = f(y + h*s1/2.0, A, B)
    s3 = f(y + h*s2/2.0, A, B)
    s4 = f(y + h*s3, A, B)
    
    return y + h/6.0*(s1 + 2.0*s2 + 2.0*s3 + s4)

def getOrbit(n, T_max, Z0):
    """ Вычисляет орбиту частицы, используя четвертый порядок
    Метод Рунге-Кутты.
    Parameters:
        n     : int. Number of iterations
        T_max : float. Stop time T=tau/t0
        Z0    : numpy-array, len(4), float.
                Position and velocities
                Z0 = [x, y, u, v]
    Returns:
        numpy-array, size(4, n). Z = [x[], y[], u[], v[]]
        Position and velocities for the orbit.
    """
    B = getB(*Z0)
    print("GR correction constant: %.2e"%(B))
    A = getA()
    
    h = T_max/float(n)
    Z = np.zeros((4, n + 1))
    Z[:, 0] = Z0

    tic = time.time()
    for i in range(0, n):
        Z[:, i + 1] = rk4step(RHS, Z[:, i], h, A, B)
    print("%.5f s, run time with %i steps."% (time.time() - tic, n))
    
    return Z

### Сохраняющиеся величины

Один из методов проверки точности нашего численного метода состоит в том, чтобы проверить, действительно ли сохраняются  величины присущие системе. В приложении мы покажем, что угловой момент на единицу массы $l = |\mathbf v\times \mathbf r|$ и энергия на единицу массы $\mathcal E$, определенные в уравнении (5), являются константами.

## Примеры

Теперь мы готовы рассмотреть некоторые примеры. Прежде чем начинать, мы создадим функцию построения графика, чтобы не повторяться. Мы построим орбиту в плоскости $xy$, эффективный потенциал и $\mathcal E$. Кроме того, мы проверим, что $\mathcal E$ и $l$ сохранены. Мы обозначим горизонт событий серым кругом.

In [ ]:
def plotOrbit(Z, lim_fact):
    """ Plots the orbit, energy and effective potential.
    Parameters:
        Z: numpy-array, size(4, n). Z = [x[], y[], u[], v[]]
           Position and velocities for the orbit.
        lim_fact: float. Axis limits are given lim_fact
           multiplied by the start position.
    """
    plt.figure(figsize=(2*fig_size, fig_size))

    rho = (Z[0, :]**2 + Z[1, :]**2)**.5
    l = Z[0, :]*Z[3, :] - Z[1, :]*Z[2, :]
    
    # Trajectory
    ax = plt.subplot(2, 2, (1, 3))
    plt.title("Orbit")
    ax.plot(Z[0, :], Z[1, :], label="Orbit")
    ax.plot(Z[0, 0], Z[1, 0], "o", label="Start")
    plt.xlabel(r"$x$")
    plt.ylabel(r"$y$")
    ax.plot(0, 0, ".", label="Origin")
    circle = plt.Circle((0, 0), radius=1, color="gray")
    ax.add_artist(circle)
    ax_lim = max(Z[0, 0], Z[1, 0])
    plt.xlim([-lim_fact*ax_lim, lim_fact*ax_lim])
    plt.ylim([-lim_fact*ax_lim, lim_fact*ax_lim])
    plt.legend()
    
    # Effective potential
    plt.subplot(2, 2, 2)
    plt.title("Effective potential")
    r = np.linspace(1.5, 20, 500)
    plt.plot(r, Veff(r, l[0]), label="Effective pot.")
    e = Veff(rho[0], l) + K(Z[:, 0:2])[0]
    plt.plot([r[0], r[-1]], [e, e], label="Energy density")
    plt.xlabel(r"$\rho$")
    plt.ylabel(r"$V$")
    
    # Energy per unit rest mass
    plt.subplot(2, 2, 4)
    plt.title("Kinetic and potential energy")
    V = Veff(rho, l)
    Ek = K(Z)
    E = V + Ek
    plt.plot(V)
    plt.plot(Ek, label=r"$\frac{1}{2}\left(\frac{dr}{d\tau}\right)^2$")
    plt.plot(E, label=r"$E$")
    plt.xlabel("Step")
    plt.ylabel("Energy density")
    plt.legend()
    print("Relative change in\n E: %.2e\n l: %.2e"%((E[0] - E[-1])/E[0], (l[0] - l[-1])/l[-1]))
    
    plt.show()

### Радиальное погружение

In [ ]:
Z0 = [0, 10, .1845, 0]
#Z0 = [0, 10, .1849, 0]
n = 5000
tau_max = 102.7
Z = getOrbit(n, tau_max, Z0)
plotOrbit(Z, 1.1)

### Прецессирующие орбиты

In [ ]:
#Z0 = [0, 20, .1, 0]
#Z0 = [0, 10, 0.2, -0.2]
#Z0 = [0, 10, .25, 0]
#Z0 = [0, 10, 0.2, -.1]
Z0 = [0, 10, .2, 0]
n = 5000
tau_max = 5000
Z = getOrbit(n, tau_max, Z0)
plotOrbit(Z, 1.1)

### Рассеиващие орбиты

In [ ]:
#Z0 = [0, 100, 0.05, -0.5]
Z0 = [0, 10, 0.2, -.25]

n = 5000
tau_max = 1000
Z = getOrbit(n, tau_max, Z0)
plotOrbit(Z, 1.1)

### Упражнения

Теперь пришло ваше время поиграть с кодами и концепциями, представленными в этом блокноте:

1. Что происходит внутри радиуса Шварцшильда? 
2. Попробуйте запустить орбиту внутри радиуса Шварцшильда. Тело может сбежать?
3. Когда частица достигает $r=0$, она улетает в бесконечность. Почему?
4. Каков период прецессирующей орбиты в приведенном выше примере в единицах СИ?
5. Что наблюдатель при $r=\infty$ будет наблюдать, когда космический аппарат приближается к горизонту событий? Как можно разрешить этот парадокс? (см. [1])

<a id="rsc"></a>
## Resources
- [1]: Hartle, J. B. (2003), Gravity: An Introduction to Einstein's General Relativity , Benjamin Cummings .

<a id="apnd"></a>
## Appendix

В этом приложении мы выведем выражения, используемые для описания движения Меркурия вокруг Солнца. Те же выражения справедливы для любой (точечной) массы вокруг сферически симметричного распределения масс. Предполагается, что читатель обладает некоторыми знаниями о Специальной теории относительности и соглашении о суммировании Эйнштейна. 
Мы будем использовать метрическое соглашение $(-+++)$ и натуральные единицы $c=G=1 (=\hbar=k_B)$.

#### До неприличия краткий обзор специальной теории относительности

Давайте начнем с того, что вспомним несколько концепций из специальной теории относительности. Рассмотрим стандартный пример, в котором инерциальная система $S'$ движется вдоль оси $x$с постоянной скоростью $v$ относительно инерциальной системы $S$. Преобразования Лоренца в этом случае задаются

$$\Delta t' = \gamma (\Delta t - v\Delta x),\quad \Delta x' = \gamma (\Delta x - v\Delta t),\quad \Delta y' = \Delta y,\quad \Delta z' = \Delta z,$$

где $\gamma = 1/\sqrt{1-v^2}$ как обычно. Обратные преобразования:

$$\Delta t = \gamma (\Delta t' + v\Delta x'),\quad \Delta x = \gamma (\Delta x' + v\Delta t'),\quad \Delta y = \Delta y',\quad \Delta z = \Delta z'.$$

Из этого мы можем вывести выражения для таких понятий, как *замедление времени*, *сокращение длины* и *одновременность*.

*Позиционный четырехвектор* $x^\alpha$ определяется как вектор-столбец с элементами $x^0 = t$, $x^1 = x$, $x^2 = y$ и $x^3 = z$. В более общем случае мы определяем четырехвектор $a^\alpha$ как четыре кортежа, которые Лоренц преобразует так же, как и $x^\alpha$. Обратите внимание, в частности, что величина $x_\alpha x^\alpha \equiv \eta_{\alpha\beta}x^{\alpha}x^{\beta}$ является инвариантом Лоренца. $\eta_{\alpha\beta} = \mathrm{diag}(-1, 1, 1, 1)$ является метрическим тензором Минковского, то есть метрическим тензором плоского пространства-времени. Это относится ко всем четырем векторам. Легко показать, что для метрики Минковского (плоского пространства) мы имеем

$$\mathrm{d} s^2 = \eta_{\alpha\beta}\mathrm{d}x^\alpha\mathrm{d}x^\beta=-\mathrm{d}t^2 + \mathrm{d}x^2+ \mathrm{d}y^2+ \mathrm{d}z^2= -1.$$

Это также работает в Общей теории относительности для общей метрики $g_{\alpha\beta}$.

Рассмотрим частицу, движущуюся вдоль оси $x$ со скоростью $v$. Пусть $t'\equiv \tau$ - собственное время частицы. *Четырехскоростная* задается

$$u^\alpha\equiv\frac{\mathrm{d}x^\alpha}{\mathrm{d}\tau} = \left(\frac{\mathrm{d} t}{\mathrm{d} \tau}, \frac{\mathrm{d} \vec x}{\mathrm{d} \tau}\right) = \gamma\left(1, \frac{\mathrm{d} \vec x}{\mathrm{d} t}\right).$$

В наших естественных единицах мы видим, что $-\mathrm{d}\tau^2 = \mathrm{d}s^2$. Это означает, что
$$u^\alpha u_\alpha = \frac{\mathrm{d} x^\alpha}{\mathrm{d}\tau}\frac{\mathrm{d} x_\alpha}{\mathrm{d}\tau} = \frac{\mathrm{d}s^2}{\mathrm{d}\tau^2} = -1.$$
Опять же, это справедливо даже для искривленного пространства-времени.

#### Метрика Шварцшильда

Нашей отправной точкой для вывода уравнений является метрика *Шварцшильда*

\begin{equation}
    \mathrm{d} s^2 = -\left(1-\frac{2GM}{c^2r}\right)(c\mathrm{d} t)^2 + \left(1-\frac{2GM}{c^2 r}\right)^{-1} \mathrm{d}r^2 + r^2\left(\mathrm{d}\theta + \sin^2\theta\mathrm{d}\phi^2\right).
\end{equation}

Эта метрика описывает кривизну пространства-времени вокруг сферически симметричного распределения массы. Для получения дополнительной информации или более глубокого понимания мы отсылаем вас, например, к [[1]](#rsc). Для простоты мы будем использовать натуральные единицы, в которых $c=G=1 (=\hbar=k_B)$. Метрика в этом случае может быть записана как

\begin{equation}
g_{\alpha\beta} = \mathrm{diag}\left(-(1-2M/r), (1-2M/r)^{-1}, r^2, r^2\sin^2\theta\right).
\end{equation}

Мы делаем следующие наблюдения (подробнее см. [[1]](#rsc) ):
1. Метрика не зависит от времени. Таким образом, существует *killing vector* $\xi^\alpha = (1, 0, 0, 0)$ связаный с симметрией смещения во времени $t$.
2. Метрика сферически симметрична. Поверхность констант $t$ и $r$ задается линейным элементом $\mathrm{d}\Sigma^2 = r^2\left(\mathrm{d}\theta^2 + \sin^2\theta\; \mathrm{d}\phi^2\right)$.
3. Метрика не зависит от $\phi$. Таким образом, существует *killing vector* $\eta^\alpha = (0,0,0,1)$, соответствующий вращательной симметрии вокруг оси $z$. Существуют дополнительные, менее интуитивные killing vector'а для метрики, но они нам здесь не понадобятся.
4. Существует координатная сингулярность при $r=2M$ и физическая сингулярность при $r=0$. Это называется радиусом Шварцшильда. Если вся масса находится в пределах $r=2M$, мы получаем *Черную дыру*, а $r=2M$ называется *Горизонтом событий*. Мы больше не будем обсуждать здесь Черные дыры.
5. Геометрия асимптотически плоская. То есть мы восстанавливаем пространство Минковского (плоское пространство-время, $g_{\alpha\beta}=\mathrm{diag}(-1, 1, 1, 1)$) за $r \gg 2G$.
6. $M$ интерпретируется как *общая масса источника кривизны* (это можно увидеть, расширив метрику для малых $GM/r$. Результатом является *статическая метрика слабого поля* с ньютоновским гравитационным потенциалом $\Phi = -GM/r$.)

#### Вывод выражений

Из наблюдений 1 и 2 следует, что величины $\xi^\alpha u_\alpha$ и $\eta^\alpha u_\alpha$ (помните, что мы используем соглашение о суммировании Эйнштейна) сохраняются, где $u^\alpha$ - четырехскоростная частица. Более конкретно, они являются постоянными вдоль геодезической. Определим

$$e \equiv -\xi^\alpha u_\alpha = g_{\alpha\beta}\xi^\alpha u^\beta = \left(1-\frac{2M}{r}\right)\frac{\mathrm{d}t}{\mathrm{d}{\tau}}$$

и

$$l\equiv \eta^\alpha u_\alpha = g_{\alpha\beta}\eta^\alpha u^\beta = r^2\sin^2\theta\frac{\mathrm{d}\phi}{\mathrm{d}{\tau}}.$$

$e$ часто называют *энергией на единицу массы покоя*, поскольку это то, к чему сводится энергия в плоском пространстве ($E=mu^t = m(\mathrm d t/\mathrm d \tau)$). Точно так же $l$ называется *угловой момент на единицу массы покоя*, потому что это то, к чему она сводится при низких скоростях. Как и в классической механике, угловой момент и энергия (на единицу массы покоя) сохраняются.

Сохранение углового момента подразумевает, что орбита лежит в заданной плоскости. Таким образом, мы выбираем $\theta=\pi/2$ и $\mathrm{d}\theta = 0$ без потери общности. В этом случае

$$ -1=g_{\alpha\beta}u^\alpha u^\beta=-\left(1-\frac{2M}{r}\right) \left(u^t\right)^2 + \left(1-\frac{2M}{r}\right)^{-1} \left(u^r\right)^2 + r^2 \left(u^\phi\right)^2.$$

Написав $u^\alpha = \mathrm d x^\alpha/\mathrm d \tau$ и используя выражения для констант $e$ и $l$, мы можем переписать:

\begin{equation}
\mathcal{E} = \frac{1}{2}\left(\frac{\mathrm{d} r}{\mathrm{d} \tau}\right)^2 + V_{\mathrm{eff}}(r),
\label{eq:energy_ein} \quad(5)
\end{equation}

где мы определили константу

$$\mathcal E \equiv \frac{e^2 - 1}{2},$$

и эффективный потенциал

$$V_\mathrm{eff}(r)\equiv -\frac{M}{r}+\frac{l^2}{2r^2}- \frac{Ml^2}{r^3}.$$

Уравнение (5) можно рассматривать как утверждение о том, что полная энергия системы равна сумме кинетической и потенциальной энергии. Если мы восстановим $c$ и $G$ и умножим на массу объекта, $m$ обе стороны уравнения (5), мы получим

\begin{equation}
E = \frac{1}{2}mu^2 + \frac{L^2}{2mr^2} - \frac{GMm}{r} - \frac{GML^2}{c^2mr^3},
\label{eq:orben} \quad(6)
\end{equation}

где $u = (\mathrm d r/\mathrm d\tau)^2$ - релятивистская скорость, $E=\mathcal E m$ - полная энергия и $L^2 = l^2m^2$ - орбитальный угловой момент. В классическом пределе ($c\to \infty$, $\mathrm d \tau\to\mathrm d t$) к хорошо известному классическому выражению для орбитальной энергии

\begin{equation}
E_\mathrm{classical} = \frac{1}{2}mv^2 + \frac{L^2}{2mr^2} - \frac{GMm}{r}.
\end{equation}

Другими словами, последний член в уравнении (6) является общей релятивистской поправкой к классическому выражению. Обратите также внимание, что мы работаем с собственным временем в уравнении (6).

Мы получаем радиальную силу, дифференцируя последние два члена в (6). В результате получается

\begin{equation}
F = -\frac{\partial }{\partial r}\left(-\frac{GMm}{r} - \frac{GML^2}{c^2mr^3}\right)=-\frac{GMm}{r^2}\left(1 + \frac{3l^2}{c^2r^2}\right).
\end{equation}